In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.DRMM.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7113.33it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3978.25it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2734112.41it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8663.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4394.28it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 874372.16it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8109.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f543427e278>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f5434656f28>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
histgram_callback = mz.dataloader.callbacks.Histogram(
    embedding_matrix, bin_size=30, hist_mode='LCH'
)

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    callbacks=[histgram_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[histgram_callback]
)

In [8]:
padding_callback = mz.models.DRMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    device='cpu',
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.DRMM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix
model.params['hist_bin_size'] = 30
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMM(
  (embedding): Embedding(30058, 300)
  (attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=30, out_features=10, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=1, bias=True)
      (1): Tanh()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9018023


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    device='cpu',
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-255 Loss-2.318]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5425 - normalized_discounted_cumulative_gain@5(0.0): 0.6073 - mean_average_precision(0.0): 0.561



[Iter-510 Loss-2.113]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5565 - normalized_discounted_cumulative_gain@5(0.0): 0.6226 - mean_average_precision(0.0): 0.5737



[Iter-765 Loss-2.032]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5546 - normalized_discounted_cumulative_gain@5(0.0): 0.6222 - mean_average_precision(0.0): 0.569



[Iter-1020 Loss-1.979]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5597 - normalized_discounted_cumulative_gain@5(0.0): 0.6275 - mean_average_precision(0.0): 0.5791



[Iter-1275 Loss-1.934]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5531 - normalized_discounted_cumulative_gain@5(0.0): 0.6211 - mean_average_precision(0.0): 0.5761



[Iter-1530 Loss-1.879]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5632 - normalized_discounted_cumulative_gain@5(0.0): 0.6278 - mean_average_precision(0.0): 0.5865



[Iter-1785 Loss-1.818]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5458 - normalized_discounted_cumulative_gain@5(0.0): 0.6146 - mean_average_precision(0.0): 0.5676



[Iter-2040 Loss-1.778]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5558 - normalized_discounted_cumulative_gain@5(0.0): 0.6136 - mean_average_precision(0.0): 0.5735



[Iter-2295 Loss-1.722]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5515 - normalized_discounted_cumulative_gain@5(0.0): 0.6078 - mean_average_precision(0.0): 0.5716



[Iter-2550 Loss-1.684]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5578 - normalized_discounted_cumulative_gain@5(0.0): 0.622 - mean_average_precision(0.0): 0.5804

Cost time: 8711.306358098984s
